In [1]:
import os.path as op
import os
import pandas as pd
import numpy as np
from copy import deepcopy

In [2]:
data_dirs = ['../data/results_D100'] # , '../data/results_D25']
files = [op.join(d, f)
         for d in data_dirs
         for f in os.listdir(d)
         if f.endswith('csv')]

In [5]:
full_results = []
for fl in files:
    tmpres = pd.read_csv(fl, index_col=0)
    fname_pieces = fl.split('_')
    dset = fname_pieces[1].split('/')[0]
    experiment = fname_pieces[2]
    n_mca = int(fname_pieces[3])
    target = fname_pieces[-4]
    dimens = fname_pieces[-2]
    preproc = fname_pieces[-3]
    random_seed = fname_pieces[-1].strip('.csv')
    resdict = {
        "Target": target,
        "Experiment": experiment,
        "Simulations": n_mca,
        "Preprocessing": preproc,
        "Dimensionality": dimens,
        "Dataset": dset,
        "rs": random_seed
    }

    for idx, r in tmpres.iterrows():
        resdict_copy = deepcopy(resdict)
        # Only add ref performance for the "full" dataset
        if r.aggregation == "ref" and n_mca != 20:
            continue
        for k, v in r.items():
            resdict_copy[k] = v
        full_results += [resdict_copy]
        del resdict_copy

df_results = pd.DataFrame.from_dict(full_results)

In [6]:
df_results

,Target,Experiment,Simulations,Preprocessing,Dimensionality,Dataset,rs,aggregation,acc,f1,p_acc,p_f1,test_acc,test_f1,p_test_acc,p_test_f1,test_mean_acc,test_mean_f1,n_models
0,age,mca,5,zgraph,fa,D100,41,none,0.643700,0.684389,0.165264,0.237424,0.75,0.827586,0.007,0.007,0.719600,0.780471,125
1,age,mca,5,zgraph,fa,D100,41,meta,0.730833,0.796438,0.090600,0.091200,0.75,0.827586,0.007,0.007,0.760000,0.835402,5
2,bmi,mca,5,graph,pca,D100,41,median,0.589167,0.570696,0.200000,0.319000,0.65,0.666667,0.065,0.054,0.600000,0.606566,5
3,bmi,mca,12,zgraph,fa,D100,41,consensus,0.564167,0.580772,0.277400,0.307200,0.65,0.631579,0.064,0.118,0.620000,0.592012,5
4,sex,mca,5,rankgraph,pca,D100,41,consensus,0.613333,0.666516,0.216400,0.260400,0.75,0.800000,0.004,0.013,0.720000,0.766227,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,bmi,mca,5,loggraph,fa,D100,41,median,0.640000,0.643004,0.180400,0.270800,0.55,0.526316,0.252,0.397,0.590000,0.559572,5
1316,bmi,mca,16,rankgraph,fa,D100,41,consensus,0.614167,0.600189,0.175600,0.272600,0.65,0.666667,0.061,0.060,0.610000,0.583482,5
1317,sex,mca,16,zgraph,pca,D100,41,none,0.615677,0.739033,0.123745,0.067928,0.65,0.774194,0.068,0.016,0.640125,0.763440,400
1318,sex,mca,16,zgraph,pca,D100,41,meta,0.539167,0.655821,0.307600,0.213400,0.65,0.758621,0.068,0.022,0.590625,0.708949,5


In [7]:
df_results.to_csv("../data/aggregation_results.csv", index=False)